In [3]:
# %pip install numpy
# %pip install pandas
# %pip install torch
# %pip install scikit-learn

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import torch.nn.functional as F

from Fourier_Transformer import LinearTransformer, create_inout_sequences
from LSTM import LSTMModel, train_model


### Data Preparation

In [7]:
df = pd.read_csv('/Users/oli/Documents/GitHub/Linear_Trans/stock_data/TSLA.csv')

df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
df.dropna(inplace=True)  # Remove NaNs
scaler = MinMaxScaler(feature_range=(-1, 1))
df['log_return'] = scaler.fit_transform(df['log_return'].values.reshape(-1,1))
data = torch.FloatTensor(df['log_return'].values).view(-1) # Convert the DataFrame to a PyTorch Tensor

seq_length = 20  # Based on how many days you want to use to predict the next day
inout_seq = create_inout_sequences(data, seq_length) # Create sequences


# Split data into train and test sets
train_size = int(len(inout_seq) * 0.80)
train_set = inout_seq[:train_size]
test_set = inout_seq[train_size:]
# Prepare DataLoader
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

### Training and Test Process

Initilize the model with 2 layers, 64 dimensions, 

In [8]:
# Instantiate the model, loss function, and optimizer
model = LinearTransformer(feature_size=1, num_layers=2, d_model=64, d_ff = 2048, num_heads=8)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
train_losses = []
test_losses = []
num_epochs = 50 

for epoch in range(num_epochs):
    model.train()
    epoch_train_loss = []
    for seq, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(seq.unsqueeze(-1))  # Adjust dimensions if necessary
        labels = labels.view(-1)  # Ensure label dimensions match output
        loss = criterion(y_pred[:, -1], labels)  # Assuming using last output for prediction
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
    
    # Calculate and store the average training loss for this epoch
    train_losses.append(np.mean(epoch_train_loss))

    # Validation or Testing phase
    model.eval()
    epoch_test_loss = []
    with torch.no_grad():
        for seq, labels in test_loader:
            y_pred = model(seq.unsqueeze(-1))
            labels = labels.view(-1)
            loss = criterion(y_pred[:, -1], labels)
            epoch_test_loss.append(loss.item())
    
    # Calculate and store the average test loss for this epoch
    test_losses.append(np.mean(epoch_test_loss))
    
    # Optional: print out loss information to monitor progress
    print(f'Epoch {epoch+1}/{num_epochs} - Training Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}')

overall_avg_train_loss = np.mean(train_losses)
overall_avg_test_loss = np.mean(test_losses)

print(f'Overall Average Training Loss: {overall_avg_train_loss:.4f}')
print(f'Overall Average Test Loss: {overall_avg_test_loss:.4f}')

# Assuming 'model' is your model instance and it has been trained
torch.save(model.state_dict(), 'transformer_fourier.pth')



/Users/oli/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/oli/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([55])) that is different to the input size (torch.Size([55, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/oli/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([46])) that is different to the input size (torch.Size([46, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same si

Epoch 1/50 - Training Loss: 2.2097, Test Loss: 0.2753
Epoch 2/50 - Training Loss: 0.2523, Test Loss: 0.5225
Epoch 3/50 - Training Loss: 0.4271, Test Loss: 0.1671
Epoch 4/50 - Training Loss: 0.0993, Test Loss: 0.1298
Epoch 5/50 - Training Loss: 0.1289, Test Loss: 0.2382
Epoch 6/50 - Training Loss: 0.1631, Test Loss: 0.1626
Epoch 7/50 - Training Loss: 0.0948, Test Loss: 0.0914
Epoch 8/50 - Training Loss: 0.0708, Test Loss: 0.0868
Epoch 9/50 - Training Loss: 0.0885, Test Loss: 0.0955
Epoch 10/50 - Training Loss: 0.0894, Test Loss: 0.0889
Epoch 11/50 - Training Loss: 0.0779, Test Loss: 0.0839
Epoch 12/50 - Training Loss: 0.0695, Test Loss: 0.0908
Epoch 13/50 - Training Loss: 0.0706, Test Loss: 0.1015
Epoch 14/50 - Training Loss: 0.0761, Test Loss: 0.1028
Epoch 15/50 - Training Loss: 0.0743, Test Loss: 0.0930
Epoch 16/50 - Training Loss: 0.0700, Test Loss: 0.0865
Epoch 17/50 - Training Loss: 0.0676, Test Loss: 0.0843
Epoch 18/50 - Training Loss: 0.0703, Test Loss: 0.0839
Epoch 19/50 - Train

### LSTM

In [15]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence, label = self.sequences[index]
        # Convert numpy arrays to torch tensors and ensure they are of type float32
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        return sequence_tensor, label_tensor

In [17]:
# Load your dataset
data = pd.read_csv('/Users/oli/Documents/GitHub/Linear_Trans/stock_data/TSLA.csv')

# Normalize data 
scaler = MinMaxScaler(feature_range=(-1, 1))
data_normalized = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Define window size
window_size = 10

# Create sequences
inout_seq = create_inout_sequences(data_normalized, window_size)

# Split data into train and test
split_ratio = 0.8
split_index = int(len(inout_seq) * split_ratio)
train_seq = inout_seq[:split_index]
test_seq = inout_seq[split_index:]


train_dataset = TimeSeriesDataset(train_seq)
test_dataset = TimeSeriesDataset(test_seq)

train_data = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_data = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [26]:
def evaluate_model(model, test_loader):
    device = next(model.parameters()).device
    model.eval()
    total_loss = []
    criterion = nn.MSELoss()
    predictions = []
    labels = []

    with torch.no_grad():
        for data, target in test_loader:
            # Ensure data and target tensors are moved to the same device as the model
            data, target = data.to(device), target.to(device)
            output = model(data)
            predictions.append(output.cpu().numpy())
            labels.append(target.cpu().numpy())
            loss = criterion(output, target)
            total_loss.append(loss.item())

    average_loss = np.mean(total_loss)
    print(f'Average Loss: {average_loss}')

In [27]:
lstm_model = LSTMModel(input_dim=1, hidden_dim=50, num_layers=1, output_dim=1)
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
train_model(lstm_model, train_data, nn.MSELoss(), lstm_optimizer, num_epochs=10)
evaluate_model(lstm_model, test_data)

Epoch 1 Loss: 0.2932860851287842
Epoch 2 Loss: 0.2234039008617401
Epoch 3 Loss: 0.24421195685863495
Epoch 4 Loss: 0.232172429561615
Epoch 5 Loss: 0.22040504217147827
Epoch 6 Loss: 0.27736034989356995
Epoch 7 Loss: 0.21010683476924896
Epoch 8 Loss: 0.1930704414844513
Epoch 9 Loss: 0.2440151572227478
Epoch 10 Loss: 0.20149116218090057
Average Loss: 0.4993472099304199


/Users/oli/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1, 1])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/oli/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([48, 1, 1])) that is different to the input size (torch.Size([48, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
